In [4]:
import os
import glob

# 1. 自动配置 Java 环境 (Colab 通常自带 Java 11 或 17)
java_paths = glob.glob("/usr/lib/jvm/java-17-openjdk-amd64") + glob.glob("/usr/lib/jvm/java-11-openjdk-amd64")
if java_paths:
    os.environ["JAVA_HOME"] = java_paths[0]
else:
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
print(f"Java 环境配置完成: {os.environ['JAVA_HOME']}")

# 2. 安装 PySpark
print("正在安装 PySpark...")
!pip install -q pyspark

# 3. 下载并解压 HBase
print("正在下载 HBase 2.4.17...")
if not os.path.exists("hbase-2.4.17"):
    !wget -q https://archive.apache.org/dist/hbase/2.4.17/hbase-2.4.17-bin.tar.gz
    !tar -xzf hbase-2.4.17-bin.tar.gz

print("✅ 第一步完成：环境准备就绪。")

Java 环境配置完成: /usr/lib/jvm/java-17-openjdk-amd64
正在安装 PySpark...
正在下载 HBase 2.4.17...
✅ 第一步完成：环境准备就绪。


In [5]:
import subprocess

# 1. 修复 HBase 配置以适配 Java 17
print("正在配置 HBase...")
hbase_env_path = "hbase-2.4.17/conf/hbase-env.sh"
with open(hbase_env_path, "a") as f:
    f.write('\nexport HBASE_DISABLE_HADOOP_CLASSPATH_LOOKUP=true\n')
    f.write('export HBASE_OPTS="$HBASE_OPTS --add-opens java.base/java.lang=ALL-UNNAMED --add-opens java.base/java.util=ALL-UNNAMED --add-opens java.base/java.util.concurrent=ALL-UNNAMED --add-opens java.base/java.io=ALL-UNNAMED"\n')

# 2. 启动 HBase
print("正在启动 HBase 服务...")
!./hbase-2.4.17/bin/start-hbase.sh

# 3. 创建表结构 (新设计)
def run_hbase_cmd(cmd):
    cmd_file = "temp_cmd.txt"
    with open(cmd_file, "w") as f:
        f.write(cmd)
    result = subprocess.run(["./hbase-2.4.17/bin/hbase", "shell", cmd_file], capture_output=True, text=True)
    return result.stdout

print("正在创建表结构...")
# 先删除旧表(如果存在)，再创建新表
# 表1: game_profile (info, metrics)
# 表2: dev_analytics (summary, product_list)
create_cmd = """
disable 'game_profile'
drop 'game_profile'
create 'game_profile', 'info', 'metrics'

disable 'dev_analytics'
drop 'dev_analytics'
create 'dev_analytics', 'summary', 'product_list'
exit
"""
run_hbase_cmd(create_cmd)

print("✅ 第二步完成：HBase 已启动，表结构已创建。")

正在配置 HBase...
正在启动 HBase 服务...
running master, logging to /content/hbase-2.4.17/bin/../logs/hbase--master-0709874da396.out
正在创建表结构...
✅ 第二步完成：HBase 已启动，表结构已创建。


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 1. 启动 Spark (带 Java 兼容参数)
print("正在启动 Spark Session...")
spark = SparkSession.builder \
    .appName("StepByStep-Design") \
    .master("local[*]") \
    .config("spark.driver.memory", "1g") \
    .config("spark.driver.extraJavaOptions", "--add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED") \
    .config("spark.executor.extraJavaOptions", "--add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED") \
    .getOrCreate()

# 2. 下载并读取数据
if not os.path.exists("games_cleaned.csv"):
    !wget -q --no-check-certificate https://raw.githubusercontent.com/evelyn-Lu7/BigData-GameDB-ETL/main/data/games_cleaned.csv

print("读取并清洗数据...")
df = spark.read.csv("games_cleaned.csv", header=True, inferSchema=True)
df_clean = df.withColumn("clean_dev", regexp_replace(col("Developers"), "[\[\]']", "")) \
             .withColumn("clean_genre", regexp_replace(col("Genres"), "[\[\]']", ""))

# 3. 准备数据 DataFrame
# A. 游戏详情 (game_profile)
df_profile = df_clean.withColumn("rowkey", reverse(col("AppID").cast("string"))) \
    .select("rowkey", "Name", "clean_dev", "clean_genre", "release_date", "clean_price", "avg_owners")

# B. 开发商统计 (dev_analytics - Summary)
df_dev_summary = df_clean.groupBy("clean_dev").agg(
    count("AppID").alias("game_count"),
    sum("avg_owners").alias("total_owners"),
    round(avg("clean_price"), 2).alias("avg_price")
)

# C. 原始数据 (用于倒排索引)
# df_clean 本身就是

print("✅ 第三步完成：Spark 数据处理完毕。")

<>:21: SyntaxWarning: invalid escape sequence '\['
<>:22: SyntaxWarning: invalid escape sequence '\['
<>:21: SyntaxWarning: invalid escape sequence '\['
<>:22: SyntaxWarning: invalid escape sequence '\['
/tmp/ipython-input-385385466.py:21: SyntaxWarning: invalid escape sequence '\['
  df_clean = df.withColumn("clean_dev", regexp_replace(col("Developers"), "[\[\]']", "")) \
/tmp/ipython-input-385385466.py:22: SyntaxWarning: invalid escape sequence '\['
  .withColumn("clean_genre", regexp_replace(col("Genres"), "[\[\]']", ""))


正在启动 Spark Session...
读取并清洗数据...
✅ 第三步完成：Spark 数据处理完毕。


In [7]:
print("正在生成 HBase 写入命令...")

# 收集少量数据演示 (Top 50)
rows_profile = df_profile.limit(50).collect()
rows_summary = df_dev_summary.orderBy(col("total_owners").desc()).limit(20).collect()
rows_raw = df_clean.limit(50).collect()

hbase_script = "batch_put_step4.txt"
with open(hbase_script, "w") as f:

    # 1. 写入 game_profile
    for row in rows_profile:
        rk = row['rowkey']
        safe_name = str(row['Name']).replace("'", "")
        safe_dev = str(row['clean_dev']).replace("'", "")

        f.write(f"put 'game_profile', '{rk}', 'info:name', '{safe_name}'\n")
        f.write(f"put 'game_profile', '{rk}', 'info:dev', '{safe_dev}'\n")
        f.write(f"put 'game_profile', '{rk}', 'info:genres', '{row['clean_genre']}'\n")
        f.write(f"put 'game_profile', '{rk}', 'metrics:price', '{row['clean_price']}'\n")
        f.write(f"put 'game_profile', '{rk}', 'metrics:owners', '{row['avg_owners']}'\n")

    # 2. 写入 dev_analytics
    # A. 写入统计
    for row in rows_summary:
        dev_name = str(row['clean_dev']).replace("'", "")
        rk_dev = dev_name.replace(" ", "_") # RowKey 处理空格

        f.write(f"put 'dev_analytics', '{rk_dev}', 'summary:game_count', '{row['game_count']}'\n")
        f.write(f"put 'dev_analytics', '{rk_dev}', 'summary:total_owners', '{row['total_owners']}'\n")

    # B. 写入倒排索引 (开发商 -> 游戏列表)
    for row in rows_raw:
        dev_name = str(row['clean_dev']).replace("'", "").replace(" ", "_")
        app_id = str(row['AppID'])
        game_name = str(row['Name']).replace("'", "")
        # 列名是 AppID, 值是 Name
        f.write(f"put 'dev_analytics', '{dev_name}', 'product_list:{app_id}', '{game_name}'\n")

print("执行写入 (Batch Put)...")
# 这里需要重新定义 run_hbase_cmd 因为之前的单元格变量可能不互通
def run_hbase_cmd_final(cmd_file):
    result = subprocess.run(["./hbase-2.4.17/bin/hbase", "shell", cmd_file], capture_output=True, text=True)
    return result.stdout

run_hbase_cmd_final(hbase_script)

print("\n=== 验证结果 ===")
print("--- game_profile (RowKey 是反转 ID) ---")
# 临时生成一个查询脚本
with open("scan_check.txt", "w") as f:
    f.write("scan 'game_profile', {LIMIT => 3}\n")
    f.write("scan 'dev_analytics', {LIMIT => 3}\n")
    f.write("exit\n")

print(run_hbase_cmd_final("scan_check.txt"))
print("✅ 第四步完成：所有数据已写入！")

正在生成 HBase 写入命令...
执行写入 (Batch Put)...

=== 验证结果 ===
--- game_profile (RowKey 是反转 ID) ---
hbase:001:0> scan 'game_profile', {LIMIT => 3}
ROW  COLUMN+CELL
 000584 column=info:dev, timestamp=2025-12-08T05:56:21.194, value=Wise Wizard Games, LLC
 000584 column=info:genres, timestamp=2025-12-08T05:56:21.220, value=Strategy
 000584 column=info:name, timestamp=2025-12-08T05:56:21.155, value=Cthulhu Realms
 000584 column=metrics:owners, timestamp=2025-12-08T05:56:21.266, value=75000
 000584 column=metrics:price, timestamp=2025-12-08T05:56:21.252, value=0.0
 00202 column=info:dev, timestamp=2025-12-08T05:56:19.497, value=Perpetual FX Creative
 00202 column=info:genres, timestamp=2025-12-08T05:56:19.542, value=Casual, Indie, Sports
 00202 column=info:name, timestamp=2025-12-08T05:56:19.418, value=Galactic Bowling
 00202 column=metrics:owners, timestamp=2025-12-08T05:56:19.705, value=10000
 00202 column=metrics:price, timestamp=2025-12-08T05:56:19.601, value=19.99
 00524 column=info:dev, timesta

In [13]:

try:
    print("📊 正在打印两张核心表的数据预览...")

    print("\n" + "="*60)
    print("1. 游戏画像表 (game_profile)")
    print("="*60)
    # truncate=False 让表格内容完整显示，不缩略
    df_profile.show(10, truncate=False)

    print("\n" + "="*60)
    print("2. 开发者榜单表 (dev_analytics)")
    print("="*60)
    df_dev_summary.orderBy(col("total_owners").desc()).show(10, truncate=False)


except NameError:
    print("❌ 报错：Spark 数据变量丢失。")
    print("请向上滚动，重新运行一下 【第三步：Spark 初始化与数据清洗】 那一格代码，然后再运行这个。")

📊 正在打印两张核心表的数据预览...

1. 游戏画像表 (game_profile)
+-------+----------------------------------+---------------------+---------------------------------------------------------------------+------------+-----------+----------+
|rowkey |Name                              |clean_dev            |clean_genre                                                          |release_date|clean_price|avg_owners|
+-------+----------------------------------+---------------------+---------------------------------------------------------------------+------------+-----------+----------+
|00202  |Galactic Bowling                  |Perpetual FX Creative|Casual, Indie, Sports                                                |2008-10-21  |19.99      |10000     |
|073556 |Train Bandit                      |Rusty Moyher         |Action, Indie                                                        |2017-10-12  |0.99       |10000     |
|0392371|Jolt Project                      |Campião Games        |Action, Adventure, Indie